In [ ]:
!pip install transformers datasets

import torch
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
import pandas as pd
import numpy as np
from transformers import TrainerCallback

In [ ]:
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast

# kogpt 모델 로드
model_name = 'skt/kogpt2-base-v2' 
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name, bos_token='</s>', eos_token='</s>', unk_token='<unk>',
    pad_token='</s>')
model = GPT2LMHeadModel.from_pretrained(model_name)


In [ ]:
# 샘플 텍스트 데이터
texts = [
    "학사제도는 대학의 학사 운영에 대한 제도입니다.",
    "수강 신청은 매 학기 시작 전 진행됩니다.",
    "성적 평가는 절대평가와 상대평가로 나뉩니다.",
    "휴학은 학칙에 따라 신청 가능합니다.",
    "복학은 지정된 기간 내에 신청해야 합니다.",
    "졸업 요건은 전공 이수 학점을 충족해야 합니다.",
    "전과 신청은 일정 기준을 만족해야 합니다.",
    "재수강은 성적 향상을 위해 가능합니다.",
    "학점 인정은 교과목의 성격에 따라 결정됩니다.",
    "수업은 온라인과 오프라인으로 진행됩니다.",
    "시험은 중간고사와 기말고사로 구성됩니다.",
    "학점 포기는 정해진 기간 내에 가능합니다.",
    "장학금 신청은 매 학기 초에 진행됩니다.",
    "교환학생 프로그램은 다양한 국가에서 운영됩니다.",
    "학생 상담은 교수님과 예약 후 가능합니다.",
    "수업 자료는 학교 웹사이트에서 확인할 수 있습니다.",
    "동아리 활동은 학업과 병행 가능합니다.",
    "등록금 납부는 지정된 은행을 통해 진행됩니다.",
    "도서관은 평일과 주말 모두 운영됩니다.",
    "캠퍼스 내 시설은 학생들에게 개방되어 있습니다.",
]

# 데이터셋 분리
train_texts = texts[:2]
test_texts = texts[2:]

# 데이터셋 생성
train_dataset = Dataset.from_dict({'text': train_texts})
test_dataset = Dataset.from_dict({'text': test_texts})


In [ ]:
#토크나이징
def tokenize_function(examples):
    model_inputs = tokenizer(examples["text"], padding='max_length', max_length=512, truncation=True)
    model_inputs["labels"] = model_inputs["input_ids"].copy()
    return model_inputs

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

In [ ]:
#파인튜닝
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    evaluation_strategy="epoch", 
    save_strategy="epoch",
    logging_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
)

trainer.train()


In [ ]:
# 샘플 Q&A 데이터
qa_data = {
    'question': [
        "수강 신청은 언제 하나요?",
        "성적 평가는 어떻게 이루어지나요?",
        "휴학 신청 방법은 무엇인가요?",
        "복학은 어떻게 하나요?",
        "졸업 요건은 무엇인가요?",
        "전과 신청은 언제 할 수 있나요?",
        "재수강은 가능한가요?",
        "학점 인정 기준은 어떻게 되나요?",
    ],
    'answer': [
        "수강 신청은 매 학기 시작 전에 합니다.",
        "성적 평가는 절대평가와 상대평가로 이루어집니다.",
        "휴학은 온라인으로 신청할 수 있습니다.",
        "복학은 학사관리팀에 문의하세요.",
        "졸업 요건은 학칙을 참고하세요.",
        "전과 신청은 2학년 말에 가능합니다.",
        "재수강은 C학점 이하일 때 가능합니다.",
        "학점 인정 기준은 교과목에 따라 다릅니다.",
    ]
}

# 데이터셋 생성
qa_dataset = Dataset.from_dict(qa_data)

# 데이터셋 분리
train_testsplit = qa_dataset.train_test_split(test_size=0.25)
train_dataset = train_testsplit['train']
test_dataset = train_testsplit['test']


In [ ]:
#토크나이징
def preprocess_function(examples):
    inputs = [q + tokenizer.eos_token + a for q, a in zip(examples['question'], examples['answer'])]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')
    model_inputs["labels"] = model_inputs["input_ids"].copy()
    
    return model_inputs


tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)

In [ ]:
#파인튜닝
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,  
    eval_dataset=tokenized_test_dataset,   
)

trainer.train()


In [ ]:
# 테스트 예시
test_questions = [
        "수강 신청은 언제 하나요?",
        "성적 평가는 어떻게 이루어지나요?",
        "휴학 신청 방법은 무엇인가요?",
        "복학은 어떻게 하나요?",
        "졸업 요건은 무엇인가요?",
        "전과 신청은 언제 할 수 있나요?",
        "재수강은 가능한가요?",
        "학점 인정 기준은 어떻게 되나요?",
]

for question in test_questions:
    inputs = tokenizer.encode(question + tokenizer.eos_token, return_tensors='pt').to(model.device)
    outputs = model.generate(inputs, max_length=512, do_sample=True, top_k=50)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = generated_text[len(question):].strip()
    print(f"질문: {question}")
    print(f"답변: {answer}")
    print("-" * 50)

